# AADT

## Imports

In [18]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError


%matplotlib inline

## Global Variables

In [19]:
IMG_SIZE = 1024
VEHICLE_DETECTION_COUNT_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/data/vehicle_counts_detection.csv'
DAILY_COUNT_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/df_aadt_week.csv'
SPEED_DATA_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/avg_mph.csv"
ROAD_WIDTH_PATH = ""
TIMESTAMP_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/timestamp.csv"
AVG_MPH_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/avg_mph.csv"

NN_MODEL_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/models/nn_model.pth"

## True Count Data
- Traffic monitoring stations for long-term traffic count data
    - Extract at same time as Satellite Image!
- How to use permanent and temporary traffic count stations

In [20]:
# Get Ground Truth Data

## Vehicle detection number
From vehicle detection model

## Road characteristics
From road characterstics pipeline

Includes:
- Road width
- Live speed data
- Directionality

## Neural Network Model

In [21]:
class CustomDataset(Dataset):
    def __init__(self):
        self.labels = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['aadt'].values.astype('float32'))
        self.vehicle_count = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['total_volume_normalised'].values.astype('float32')).unsqueeze(1) # Training
        #self.speed_data = pd.read_csv(SPEED_DATA_PATH) 
        #self.road_width = pd.read_csv(ROAD_WIDTH_PATH)
        self.hour = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['hour'].values.astype('float32')).unsqueeze(1)
        self.avg_mph = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['avg_mph'].values.astype('float32')).unsqueeze(1)
        self.day = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['day'].values.astype('float32')).unsqueeze(1)
        self.month = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['month'].values.astype('float32')).unsqueeze(1)

        self.x = torch.concat((self.vehicle_count, self.avg_mph, self.day, self.month, self.hour), dim=-1)
        self.y = self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

custom_data = CustomDataset()
train_split = 0.8
train_data, test_data = random_split(custom_data, [train_split, 1-train_split])

In [23]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 5),
            nn.Linear(5,20),
            nn.Linear(20,1)
            #nn.ReLU(),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [24]:
nn_model = NeuralNetwork()
nn_model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [25]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

nn_model.apply(init_weights)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [26]:
learning_rate = 1e-1
batch_size = 1
epochs = 1
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

MAPE = MeanAbsolutePercentageError()

### DataLoaders

In [27]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)

In [28]:
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)

### Training & Testing

In [29]:
def run_epoch(ep_id, action, loader, model, optimizer, criterion):
    losses = [] # Keep list of accuracies to track progress
    is_training = action == "train" # True when action == "train", else False 

    # Looping over all batches
    for batch_idx, batch in enumerate(loader): 
            x, y = batch
            #print("x: {}".format(x))
            #print("y: {}".format(y))

            # Resetting the optimizer gradients
            optimizer.zero_grad()

            # Setting model to train or test
            with torch.set_grad_enabled(is_training):
                
                # Feed batch to model
                logits = nn_model(x).squeeze(1)
                print("logits: {}".format(logits))

                # Calculate the loss based on predictions and real labels
                loss = criterion(logits, y)
                print("loss: {}".format(loss))
                mape_loss = MAPE(logits, y)

                # If training, perform backprop and update weights
                if is_training:
                    loss.backward()
                    optimizer.step()

                # Append current batch accuracy
                losses.append(mape_loss.detach().numpy())

                # Print some stats every 50th batch 
                if batch_idx % 50 == 0:
                    print(f"{action.capitalize()}ing, Epoch: {ep_id+1}, Batch {batch_idx}: Loss = {loss.item()}")
    # Return accuracies to main loop                 
    return losses

In [30]:
def main(epochs, train_dl, test_dl, model, optimizer, criterion):

    # Keep lists of accuracies to track performance on train and test sets
    train_losses = []
    test_losses = []

    # Looping over epochs
    for epoch in range(epochs):
        
        # Looping over train set and training
        train_loss = run_epoch(epoch, "train", train_dl, model, optimizer, criterion)

        # Looping over test set
        test_loss = run_epoch(epoch, "test", test_dl, model, optimizer, criterion) 

        # Collecting stats
        train_losses += train_loss
        test_losses += test_loss         
            
    return train_losses, test_losses

In [31]:
train_losses, test_losses = main(epochs=epochs, train_dl=train_dataloader, test_dl=test_dataloader, model=nn_model, optimizer=optimizer, criterion=loss_fn)

logits: tensor([-27.9224], grad_fn=<SqueezeBackward1>)
loss: 334914304.0
Training, Epoch: 1, Batch 0: Loss = 334914304.0
logits: tensor([22.4050], grad_fn=<SqueezeBackward1>)
loss: 333074848.0
logits: tensor([66.4024], grad_fn=<SqueezeBackward1>)
loss: 331470816.0
logits: tensor([141.3888], grad_fn=<SqueezeBackward1>)
loss: 328745984.0
logits: tensor([272.9131], grad_fn=<SqueezeBackward1>)
loss: 323993888.0
logits: tensor([512.6834], grad_fn=<SqueezeBackward1>)
loss: 315419680.0
logits: tensor([822.3222], grad_fn=<SqueezeBackward1>)
loss: 304517152.0
logits: tensor([1259.9628], grad_fn=<SqueezeBackward1>)
loss: 289434656.0
logits: tensor([2052.2590], grad_fn=<SqueezeBackward1>)
loss: 263104080.0
logits: tensor([2561.1357], grad_fn=<SqueezeBackward1>)
loss: 246854576.0
logits: tensor([3877.5342], grad_fn=<SqueezeBackward1>)
loss: 207222016.0
logits: tensor([5276.5591], grad_fn=<SqueezeBackward1>)
loss: 168900784.0
logits: tensor([7895.9409], grad_fn=<SqueezeBackward1>)
loss: 107677992.0

### Loss Curve Plot

In [32]:
px.line(train_losses)

In [33]:
px.line(test_losses)

### Random test sample

In [34]:
preds = []
ground_truth = []
vehicle_counts = []


for i in range(20):
    random_idx = np.random.randint(0,len(test_data))
    x, y = test_data[random_idx]
    vehicle_counts.append(x[0])
    ground_truth.append(float(y))
    pred_y = float(nn_model(x)[0])
    preds.append(pred_y)

df = pd.DataFrame({'vehicle_count': vehicle_counts, 'ground_truth': ground_truth, 'predictions': preds})

px.scatter(df, x='vehicle_count', y=['ground_truth', 'predictions'])

## Save Model

In [35]:
torch.save(nn_model.state_dict(), NN_MODEL_PATH)